In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
member = pd.read_csv('member_data.csv')
product = pd.read_csv('product.csv')

# 이상치처리

## 비조합원 삭제

In [3]:
member = member.drop(4502)
member = member.reset_index(drop=True)

In [4]:
product = product.loc[product['회원번호'] != 1670400000]

## 구매수량 0 삭제
65행 삭제

In [5]:
product = product.drop(product.loc[product['구매수량'] == 0].index).reset_index(drop=True)

## 연령, 성별 결측인 회원정보 및 구매정보 삭제
고객정보 3행 삭제  
구매정보 20행 삭제

In [6]:
null_customer = member.loc[(member['연령'].isnull()) | (member['성별'].isnull()), '회원번호'].values

In [7]:
product.drop(product.loc[product['회원번호'] == null_customer[0]].index, inplace=True)
product.drop(product.loc[product['회원번호'] == null_customer[1]].index, inplace=True)
product.drop(product.loc[product['회원번호'] == null_customer[2]].index, inplace=True)

In [8]:
member = member.loc[~member['연령'].isnull()].reset_index(drop=True)

In [9]:
member = member.loc[~member['성별'].isnull()].reset_index(drop=True)

## 공급일자 형 변경
object -> datetime

In [10]:
product['공급일자'] = pd.to_datetime(product['공급일자'])

## 반품원거래일자 형 변경
float -> datetime

In [11]:
product.loc[product['반품_원거래일자'].notnull(), '반품_원거래일자'] = product.loc[product['반품_원거래일자'].notnull(), '반품_원거래일자'].astype('int')

In [12]:
product['반품_원거래일자'] = pd.to_datetime(product['반품_원거래일자'], format='%Y%m%d')

## 원거래일자 결측값 대체

424행 대체  
NaN -> 공급일자

In [13]:
product.loc[(product['구매수량'] < 0) & (product['반품_원거래일자'].isnull()), '반품_원거래일자'] = product.loc[(product['구매수량'] < 0) & (product['반품_원거래일자'].isnull()), '공급일자']

## 모바일알림여부 수정

. -> 미수신

In [14]:
member['모바일알람여부'] = member['모바일알람여부'].str.replace('.', '미수신')

# 파생변수 생성

## 제품 정보 파일

### 요일 생성

In [15]:
product['요일'] = product['공급일자'].dt.day_name()

### 주차 생성

In [16]:
product['주차'] = product['공급일자'].dt.weekofyear

### 주중/주말 생성

In [17]:
product['주중'] = '주중'
product.loc[product['요일'] == 'Saturday', '주중'] = '주말'
product.loc[product['요일'] == 'Sunday', '주중'] = '주말'

### 계절 생성
12 ~ 2 : 겨울  
3 ~ 5 : 봄  
6 ~ 8 : 여름  
9 ~ 11 : 가을

In [18]:
product['계절'] = '여름'

product.loc[product['공급일자'].dt.month == 1, '계절'] = '겨울'
product.loc[product['공급일자'].dt.month == 2, '계절'] = '겨울'
product.loc[product['공급일자'].dt.month == 3, '계절'] = '봄'
product.loc[product['공급일자'].dt.month == 4, '계절'] = '봄'
product.loc[product['공급일자'].dt.month == 5, '계절'] = '봄'
product.loc[product['공급일자'].dt.month == 9, '계절'] = '가을'
product.loc[product['공급일자'].dt.month == 10, '계절'] = '가을'

### 구매시간대 생성
가게 운영 시간은 오전 8시부터 오후 9시까지

08 ~ 12 아침  
13 ~ 17 점심  
18 ~ 20 저녁

In [19]:
product['구매시간대'] = '아침'
hour = product['구매시각'].str.partition(':')[0]

product.loc[hour == '13', '구매시간대'] = '점심'
product.loc[hour == '14', '구매시간대'] = '점심'
product.loc[hour == '15', '구매시간대'] = '점심'
product.loc[hour == '16', '구매시간대'] = '점심'
product.loc[hour == '17', '구매시간대'] = '점심'
product.loc[hour == '18', '구매시간대'] = '저녁'
product.loc[hour == '19', '구매시간대'] = '저녁'
product.loc[hour == '20', '구매시간대'] = '저녁'

## 고객 정보 파일

### 연령대 생성
15 ~ 24  
25 ~ 34  
35 ~ 44  
45 ~ 54  
55 ~ 64  
65 이상

In [20]:
member['연령대'] = '34 이하'

member.loc[(member['연령'] >= 36) & (member['연령'] <= 45), '연령대'] = '35 ~ 44'
member.loc[(member['연령'] >= 46) & (member['연령'] <= 55), '연령대'] = '45 ~ 54'
member.loc[(member['연령'] >= 56) & (member['연령'] <= 65), '연령대'] = '55 ~ 64'
member.loc[(member['연령'] > 65), '연령대'] = '65 이상'

### 구 생성

In [21]:
member['구'] = member['주소'].str.partition('-').loc[:, 0]

### 동 생성

In [22]:
member['동'] = member['주소'].str.partition('-').loc[:, 2]

### 반품횟수 생성
반품한 물품의 갯수이다.

In [23]:
temp = pd.DataFrame(product.loc[product['구매수량'] < 0].groupby('회원번호').count()['물품명']).reset_index()
temp.columns = ['회원번호', '반품횟수']
member = pd.merge(member, temp, how='left', on='회원번호')
member['반품횟수'] = member['반품횟수'].fillna(0)
member['반품횟수'] = member['반품횟수'].astype('int')

### 방문횟수 생성

In [24]:
temp = pd.pivot_table(index=['공급일자', '회원번호'], aggfunc='count', data=product).reset_index()['회원번호']
member = pd.merge(member, pd.Series(Counter(temp), name='방문횟수'), how='left', left_on='회원번호', right_index=True)

## 이상치처리 - 구매없이 반품만 존재하는 구매정보 삭제
반품횟수 생성 이후 진행할 것!!
2331 건 삭제

In [25]:
for i in member.loc[member['반품횟수'] >= 1, '회원번호'].values:
    pt = pd.pivot_table(index='물품명', values='구매수량', aggfunc='sum', data=product[product['회원번호'] == i]).reset_index()
    for j in pt.loc[pt['구매수량'] < 0, '물품명'].values:
        product.drop(product.loc[(product['회원번호'] == i) & (product['물품명'] == j)].index, inplace=True)

In [26]:
product = product.reset_index(drop=True)

## 이상치처리 - 반품내역만 있는 회원정보 삭제

In [27]:
drop_list = list(set(member['회원번호'].unique()) - set(product['회원번호'].unique()))

In [28]:
for i in drop_list:
    member.drop(member[member['회원번호'] == i].index, inplace=True)

In [29]:
member = member.reset_index(drop=True)

### 반품횟수 갱신
이상치 처리 이후 갱신필요

In [30]:
member.drop('반품횟수', axis=1, inplace=True)
temp = pd.DataFrame(product.loc[product['구매수량'] < 0].groupby('회원번호').count()['물품명']).reset_index()
temp.columns = ['회원번호', '반품횟수']
member = pd.merge(member, temp, how='left', on='회원번호')
member['반품횟수'] = member['반품횟수'].fillna(0)
member['반품횟수'] = member['반품횟수'].astype('int')

### 총구매금액 생성

In [31]:
temp = pd.pivot_table(index='회원번호', aggfunc='sum', values='구매금액', data=product).reset_index()
member = pd.merge(member, temp, on='회원번호', how='outer')
member['구매금액'] = member['구매금액'].fillna(0)
member.rename(columns={'구매금액':'총구매금액'}, inplace=True)

### 1회평균구매금액 생성

In [32]:
member['1회평균구매금액'] = member['총구매금액'] / member['방문횟수']
member['1회평균구매금액'] = member['1회평균구매금액'].round()

### 주별평균구매금액 생성

In [33]:
member['주별평균구매금액'] = (member['총구매금액'] / 44).astype('int')

### 첫방문일, 최종방문일 생성

In [34]:
for cn in member['회원번호']:
    sort_values = product.loc[product['회원번호'] == cn, '공급일자'].sort_values()
    member.loc[member['회원번호'] == cn, '첫방문일'] = sort_values.iloc[0]
    member.loc[member['회원번호'] == cn, '최종방문일'] = sort_values.iloc[-1]

member['첫방문일'] = member['첫방문일'].dt.dayofyear
member['최종방문일'] = member['최종방문일'].dt.dayofyear

### 미방문기간 생성

304가 10월 31일이다.

In [35]:
member['미방문기간'] = 304 - member['최종방문일']

### 방문주기 생성

In [36]:
member['방문주기'] = (member['최종방문일'] - member['첫방문일']) / member['방문횟수']

In [37]:
member['방문주기'] = member['방문주기'].round(1)

### 최대매출매장

In [38]:
temp = pd.pivot_table(data=product, index='회원번호', columns='구매매장', values='구매금액', aggfunc='sum').fillna(0).astype('int')

In [39]:
temp['최대매출매장'] = temp.idxmax(axis=1)

In [40]:
member = pd.merge(member, temp['최대매출매장'], left_on='회원번호', right_index=True)

### 구매증감율

In [41]:
product['공급일자'] = pd.to_datetime(product['공급일자']).dt.dayofyear

In [42]:
date = member.loc[member['방문주기'] != 0, '첫방문일'] + (member.loc[member['방문주기'] != 0, '최종방문일'] - member.loc[member['방문주기'] != 0, '첫방문일']) / 2

In [43]:
date.name = '중앙방문일'

In [44]:
df = pd.concat([member.loc[member['방문주기'] != 0, '회원번호'], date], axis=1)

In [45]:
member['구매증감율'] = 0

In [47]:
for number, date in zip(df['회원번호'], df['중앙방문일']):
    first = product.loc[(product['공급일자'] < date) & (product['회원번호'] == number), '구매금액'].sum()
    last = product.loc[(product['공급일자'] >= date) & (product['회원번호'] == number), '구매금액'].sum()
    with np.errstate(divide='ignore'):
        member.loc[member['회원번호'] == number, '구매증감율'] = np.where(first != 0, (last - first) / first, 1)

In [48]:
member['구매증감율'] = member['구매증감율'].round(3)

### 연령, 1회평균구매금액 형 변환
float -> int

In [49]:
member[['연령', '1회평균구매금액']] = member[['연령', '1회평균구매금액']].astype(int)

 # 최종확인

In [50]:
member

,회원번호,조합원상태,주소,성별,연령,배송서비스신청여부,모바일알람여부,Gold_member,연령대,구,...,반품횟수,총구매금액,1회평균구매금액,주별평균구매금액,첫방문일,최종방문일,미방문기간,방문주기,최대매출매장,구매증감율
0,1000012800,정상회원,수지구-풍덕천동,여,58,미신청,미수신,normal,55 ~ 64,수지구,...,0,7658,7658,174,165,165,139,0.0,매장B,0.000
1,1000103744,정상회원,처인구-고림동,여,34,미신청,미수신,normal,34 이하,처인구,...,0,36948,36948,839,286,286,18,0.0,매장C,0.000
2,1000122304,정상회원,광주-기타,여,38,미신청,미수신,normal,35 ~ 44,광주,...,0,142831,9522,3246,104,304,0,13.3,매장C,0.859
3,1000163136,정상회원,수지구-동천동,여,73,미신청,미수신,normal,65 이상,수지구,...,0,138892,34723,3156,258,304,0,11.5,매장A,-0.503
4,1000281920,정상회원,수지구-풍덕천동,여,42,신청,미수신,normal,35 ~ 44,수지구,...,0,632554,22591,14376,2,279,25,9.9,매장C,-0.305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11537,999682432,탈퇴,수지구-동천동,여,47,미신청,미수신,normal,45 ~ 54,수지구,...,0,39246,39246,891,8,8,296,0.0,매장A,0.000
11538,999721408,정상회원,수지구-동천동,여,43,미신청,미수신,normal,35 ~ 44,수지구,...,0,43457,14486,987,178,264,40,28.7,매장A,1.847
11539,999738112,정상회원,수지구-성복동,여,67,미신청,미수신,normal,65 이상,수지구,...,0,152960,16996,3476,28,291,13,29.2,매장D,-0.318
11540,999754816,정상회원,수지구-상현동,여,44,신청,미수신,normal,35 ~ 44,수지구,...,0,641334,30540,14575,10,295,9,13.6,매장B,-0.139


In [52]:
member.to_csv('member_data_preprocessed.csv', index=False, encoding='utf-8')

In [51]:
product

,공급일자,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,구매금액,구매매장,반품_원거래일자,구매시각,요일,주차,주중,계절,구매시간대
0,2,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,22207,매장C,NaT,10:04,Tuesday,1,주중,겨울,아침
1,2,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,4403,매장C,NaT,10:09,Tuesday,1,주중,겨울,아침
2,2,743192512,정상회원,채소,양념채소,생강,생강(300g),1.0,7083,매장C,NaT,10:12,Tuesday,1,주중,겨울,아침
3,2,176912064,정상회원,축산물,유제품,우유/산양유,유기농저지방우유(종이팩/900ml),1.0,7849,매장C,NaT,10:23,Tuesday,1,주중,겨울,아침
4,2,87592064,정상회원,양념/가루,가루,양념가루,카레(매운맛: 100g),3.0,15507,매장C,NaT,10:29,Tuesday,1,주중,겨울,아침
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290887,304,1110819712,정상회원,채소,뿌리채소,무,무(0.7~1kg미만),1.0,2680,매장A,NaT,13:26,Wednesday,44,주중,가을,점심
290888,304,1517953728,정상회원,채소,양념채소,쪽파,쪽파(500g),1.0,4212,매장A,NaT,10:58,Wednesday,44,주중,가을,아침
290889,304,713691392,정상회원,반찬,밑반찬,조림,우엉조림(반찬용/150g),1.0,7658,매장A,NaT,14:36,Wednesday,44,주중,가을,점심
290890,304,247926336,정상회원,간식,빵,건강곡물빵,감자바게트(180g)/매장,1.0,5169,매장A,NaT,16:23,Wednesday,44,주중,가을,점심


In [53]:
product.to_csv('product_preprocessed.csv', index=False, encoding='utf-8')